# Assignment 4

In [3]:
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
from spacy.tokens import Doc
import json
from datasets import load_dataset

In [4]:
df = pd.read_csv("ner.csv")
df.head()

,Sentence #,Sentence,POS,Tag
0,Sentence: 1,Thousands of demonstrators have marched throug...,"['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 2,Families of soldiers killed in the conflict jo...,"['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,Sentence: 3,They marched from the Houses of Parliament to ...,"['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,Sentence: 4,"Police put the number of marchers at 10,000 wh...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 5,The protest comes on the eve of the annual con...,"['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [5]:
df = df.drop(["Tag"], axis=1)

In [6]:
df.head(5)

,Sentence #,Sentence,POS
0,Sentence: 1,Thousands of demonstrators have marched throug...,"['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'..."
1,Sentence: 2,Families of soldiers killed in the conflict jo...,"['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ..."
2,Sentence: 3,They marched from the Houses of Parliament to ...,"['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ..."
3,Sentence: 4,"Police put the number of marchers at 10,000 wh...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ..."
4,Sentence: 5,The protest comes on the eve of the annual con...,"['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D..."


In [7]:
df['Sentence #'].value_counts()

Sentence #
Sentence: 1        1
Sentence: 31978    1
Sentence: 31969    1
Sentence: 31970    1
Sentence: 31971    1
                  ..
Sentence: 15989    1
Sentence: 15990    1
Sentence: 15991    1
Sentence: 15992    1
Sentence: 47959    1
Name: count, Length: 47959, dtype: int64

In [8]:
sample_df = df.head(210).copy()

## SpaCy en_core_web_sm

In [10]:
# Function to get Doccano-style entities
nlp = spacy.load("en_core_web_sm")

def get_spacy_entities(text):
    doc = nlp(text)
    return [[ent.start_char, ent.end_char, ent.label_] for ent in doc.ents]

# Create Doccano-ready columns
sample_df["text"] = sample_df["Sentence"]
sample_df["label"] = sample_df["Sentence"].apply(get_spacy_entities)

# Only keep relevant columns
spacy_df = sample_df[["text", "label"]]

In [11]:
def spacy_export(df, output_path="spacy_ner.jsonl"):
    with open(output_path, "w", encoding="utf-8") as f:
        for _, row in df.iterrows():
            text = row["text"]
            labels = row["label"] if isinstance(row["label"], list) else []
            json.dump({"text": text, "label": labels}, f)
            f.write("\n")

# Usage
spacy_export(spacy_df)


# STOP HERE


In [13]:
# Retrieve the entity label names from the dataset
labels = dataset["train"].features["ner_tags"].feature.names

# Function to convert numerical NER tags into human-readable labels
def decode_ner_tags(example):
    example["ner_labels"] = [labels[tag] for tag in example["ner_tags"]]
    return example

# Apply the function to all data splits
dataset = dataset.map(decode_ner_tags)

NameError: name 'dataset' is not defined

In [ ]:
print("Entity Types in the Dataset:", labels)


In [ ]:
train_dataset = dataset["train"]
df = train_dataset.to_pandas()

In [ ]:
# Count unique document IDs
num_documents = df["doc_id"].nunique()
print(f"Number of unique documents: {num_documents}")

In [ ]:
import spacy
from spacy.tokens import Doc
from spacy import displacy

# Load a blank spaCy model
nlp = spacy.blank("en")

# Function to visualize Named Entities
def visualize_ner(example):
    tokens = example["tokens"]
    labels = example["ner_labels"]
    start_chars = example["start_char"]
    end_chars = example["end_char"]

    # Convert entity positions into spaCy's expected format
    ents = []
    for start, end, label in zip(start_chars, end_chars, labels):
        if label != "O":  # Ignore "O" (Outside) labels
            ents.append(nlp.make_doc(example["sentence"]).char_span(start, end, label=label))

    # Create a spaCy Doc object
    doc = nlp.make_doc(example["sentence"])

    # Assign named entities (filter out None values)
    doc.ents = [ent for ent in ents if ent is not None]

    # Render visualization in Jupyter Notebook
    displacy.render(doc, style="ent", jupyter=True)

# Display the first example with highlighted named entities
visualize_ner(dataset["train"][1])

In [ ]:
# Function to apply the pretrained model and display structured results
def evaluate_ner_predictions(example):
    text = example["sentence"]
    doc = nlp_core(text)  # Use pretrained model
    predictions = [(ent.text, ent.label_) for ent in doc.ents]  # Extract model-predicted entities

    # Compare with dataset labels
    expected_entities = list(zip(example["tokens"], example["ner_labels"]))

    # Convert to DataFrame for better readability
    df_results = pd.DataFrame(predictions, columns=["Text Span", "Model Prediction"])
    df_expected = pd.DataFrame(expected_entities, columns=["Text Span", "Expected Label"])

    # Merge for side-by-side comparison
    results_df = df_results.merge(df_expected, on="Text Span", how="outer").fillna("N/A")

    print(results_df)

# Test on a sample
evaluate_ner_predictions(dataset["train"][1])

In [ ]:
# Function to process text and extract entities
def get_entities_from_text(text):
    # Process the text with spaCy model
    doc = nlp(text)
    # Return a list of tuples with entity text and label
    return [(ent.text, ent.label_) for ent in doc.ents]

# Manually annotated sentences
manual_annotations = [
    ("2011-05-04T07:24:07Z -- http://www.bloomberg.com/news/2011-05-04/bbva-is-said-to-announce-falling-profit-on-thursday.html", []),
    ("Net income probably declined to 1.08 billion euros ($1.6 billion) from 1.24 billion euros a year earlier, according to the average estimate in a Bloomberg survey of six analysts.", [("Net income", "MONEY"), ("euros", "MONEY"), ("Bloomberg", "ORG"), ("six analysts", "ORG")]),
    ("The Bilbao, Spain-based lender is scheduled to report first-quarter results tomorrow before the stock market opens in Spain.", [("Bilbao", "GPE"), ("Spain", "GPE")]),
    ("Spanish banks have been hurt by weakening credit demand and souring loans in a domestic economy that’s still struggling to emerge from recession.", [("Spanish banks", "ORG")]),
    ("BBVA, led by Chairman Francisco Gonzalez, completed its $5.8 billion purchase of a 24.9 percent stake in Turkiye Garanti Bankasi AS (GARAN) in March as it expands outside Spain, adding Turkey to other emerging markets such as Mexico.", [("BBVA", "ORG"), ("Francisco Gonzalez", "PERSON"), ("$5.8 billion", "MONEY"), ("Turkiye Garanti Bankasi AS (GARAN)", "ORG"), ("March", "TIME"), ("Spain", "GPE"), ("Turkey", "GPE"), ("Mexico", "GPE")]),
    ("“The news from Spain won’t be good, but there probably won’t be nasty surprises while Mexico is showing interesting growth,” said Pablo Garcia, head of equities at Oddo Sociedad de Valores in Madrid.", [("Spain", "GPE"), ("Mexico", "GPE"), ("Pablo Garcia", "PERSON"), ("Oddo Sociedad de Valores", "ORG"), ("Madrid", "GPE")]),
    ("“All the big Spanish companies, and that includes BBVA, are at pains now to show that Spain is a diminishing part of their business.” BBVA shares have gained 14 percent this year, compared with a 2.9 percent advance in the 48-member Bloomberg Europe Banks and Financial Services Index.", [("Spanish companies", "ORG"), ("BBVA", "ORG"), ("Spain", "GPE"), ("BBVA", "ORG"), ("Bloomberg", "ORG"), ("Europe", "GPE")]),
    ("Banco Santander SA (SAN), Spain’s largest bank, which on April 28 reported a 5 percent drop in first-quarter profit, has risen 5.3 percent in 2011.", [("Banco Santander SA (SAN)", "ORG"), ("Spain", "GPE"), ("April 28", "TIME"), ("2011", "TIME")]),
    ("Mexican Gains Profit from BBVA’s Spain-dominated Iberian business may have dropped 33 percent from a year earlier to 394 million euros, according to estimates by Banco BPI SA (BPI) analyst Carlos Joaquim Peixoto.", [("BBVA", "ORG"), ("Spain", "GPE"), ("Iberian business", "ORG"), ("Banco BPI SA (BPI)", "ORG"), ("Carlos Joaquim Peixoto", "PERSON")]),
    ("Spain and Portugal together accounted for 45 percent of group profit in 2010.", [("Spain", "GPE"), ("Portugal", "GPE"), ("2010", "TIME")])
]

# Retrieve 10 random samples from the dataset
sample_data = dataset["train"].select(range(10))

# Store annotations and predictions
annotations = []

for i, example in enumerate(sample_data):
    text = example["sentence"]
    
    # Get the manual annotations for the text
    manual_annotation = manual_annotations[i][1]
    
    # Get the model's predictions
    model_prediction = get_entities_from_text(text)
    
    # Store both manual annotations and model predictions
    annotations.append({
        "text": text,
        "manual_label": manual_annotation,
        "model_prediction": model_prediction
    })

# Convert to DataFrame for easy viewing
df_manual = pd.DataFrame(annotations)
print(df_manual)